# Memory 관리에 관한 실습코드입니다.

In [5]:
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

# Chat 모델은 llama-3-ko 7b모델을 로컬로 사용해봤습니다.

In [8]:
from langchain_openai import ChatOpenAI
# chat = ChatOpenAI(model = "gpt-3.5-turbo-1106")
chat = ChatOpenAI(base_url="http://localhost:1234/v1",
                  api_key="lm-studio",
                  model = "cognitivecomputations/dolphin-2.9-llama3-8b-gguf",
                  temperature=0.0)

In [10]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "you are a helpful assistant. Answer all questions to the best of your ability",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | chat

chain.invoke(
    {
        "messages": [
            HumanMessage(
                content = "Translate this sentence from English to French: I love programming"
            ),
            AIMessage(content = "J'adore la programmation."),
            HumanMessage(content = "What did you just say?"),
        ],
    }
).content

'I said, "J\'adore la programmation," which translates to "I love programming" in English.\n'

# Chat history
- 메시지를 직접 배열로 저장하고 전달하는 것도 괜찮지만, LangChain의 내장 메시지 기록 클래스를 사용하여 메시지를 저장하고 로드할 수도 있습니다. 
- 이 클래스의 인스턴스는 영속적 저장소에서 채팅 메시지를 저장하고 로드하는 역할을 합니다. 
- LangChain은 다양한 제공업체와 통합되어 있습니다 
- 여기에서 통합 목록을 볼 수 있습니다 
- 하지만 이 데모에서는 일회성 데모 클래스를 사용할 것입니다.


In [11]:
from langchain.memory import ChatMessageHistory
chat_history = ChatMessageHistory()
chat_history.add_user_message(
    "Tlanslate this sentence from English to French: I love programming"
)
chat_history.add_ai_message("J'adore la programmation.")
chat_history.messages

[HumanMessage(content='Tlanslate this sentence from English to French: I love programming'),
 AIMessage(content="J'adore la programmation.")]

In [17]:
input_1 = "Translate this sentence from English to French: I love programming."

chat_history = ChatMessageHistory()
chat_history.add_user_message(input_1)
response = chain.invoke(
    {
    "messages": chat_history.messages,
    }
)
chat_history.add_ai_message(response) # input_1에 대한 response를 chat_history에 추가

input_2 = "What did I just ask you?"

chat_history.add_user_message(input_2)
chain.invoke(
    {
        "messages":chat_history.messages,
    }
).content

'You asked me to translate an English sentence into French, which was "I love programming." The translation is "Je aime programmer."\n\nThen, you asked what did I just ask you, and now you\'re asking for a response.'

# # 자동 히스토리 관리
- 이전 예제들은 명시적으로 체인에 메시지를 전달합니다.
- 이것은 완전히 수용 가능한 방법이지만, 새로운 메시지를 외부에서 관리해야 합니다.
- LangChain은 이 프로세스를 자동으로 처리할 수 있는 LCEL 체인을 위한 RunnableWithMessageHistory라는 래퍼도 포함하고 있습니다.
- 
- 이 작동 방식을 보여주기 위해, 위의 프롬프트를 약간 수정하여 채팅 히스토리 이후에 HumanMessage 템플릿을 채우는 최종 입력 변수를 사용하도록 합시다.
- 이는 현재 메시지 이전의 모든 메시지를 포함하는 chat_history 매개변수를 예상할 것을 의미합니다:


